# Nettoyage des données Journalieres

In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')
z=557
l=33.80

**Nettoyage des données d'ENAM Meknès**

In [2]:
EnamMksJr=pd.read_csv(r'ENAM Meknes journalier 09 11 2016 au 07 02 2022.csv',sep=';')

In [3]:
EnamMksJr.isnull().sum()

Date/heure                 0
Température moy            0
Température max            0
Température min            0
Rayonnement solaire moy    0
Humidité moy               0
Humidité max               0
Humidité min               0
Precipitation Somme        0
Vitesse du vent  moy       2
Vitesse du vent  max       2
ETP quotidien [mm]         6
dtype: int64

**Supprimer les lignes dans la vitesse du vent moy et max sont null**

In [4]:
EnamMksJr=EnamMksJr.dropna(subset=["Vitesse du vent  moy","Vitesse du vent  max","ETP quotidien [mm]"])

**Supprimer la ligne si Humidité max < Humidité min ou viteese de vent < 0 ou Température max < Température Min**

In [5]:
for i in EnamMksJr.index:
    if(EnamMksJr["Humidité max"][i]<EnamMksJr["Humidité min"][i]):
        EnamMksJr.drop(i, axis=0, inplace=True)
        EnamMksJr.reset_index(drop=True, inplace=True)
    elif(EnamMksJr["Vitesse du vent  moy"][i]<0):
        EnamMksJr.drop(i, axis=0, inplace=True)
        EnamMksJr.reset_index(drop=True, inplace=True)
    elif(EnamMksJr["Température max"][i]<EnamMksJr["Température min"][i]):
        EnamMksJr.drop(i, axis=0, inplace=True)
        EnamMksJr.reset_index(drop=True, inplace=True)

**Convertir la date du jour au nombre de jour dans l'année**

In [6]:
for i in EnamMksJr.index:
    date_j = dt.strptime(EnamMksJr["Date/heure"][i], '%d/%m/%Y %H:%M')
    j = int(date_j.strftime("%j"))
    EnamMksJr["Date/heure"][i] = j

**Nettoyage des données journaliere d'ANAM Berkane**

In [7]:
AnamBerkane=pd.read_csv(r'ANAM Berkane journalier restrint 01 08 2017 au 07 02 2022.csv',sep=';')

In [8]:
AnamBerkane.isnull().sum()

Date/heure                 0
Température moy            0
Température max            0
Température min            0
Rayonnement solaire moy    0
Humidité moy               0
Humidité max               0
Humidité min               0
Precipitation Somme        0
Vitesse du vent  moy       0
Vitesse du vent  max       0
Vitesse Max max            0
ETP quotidien [mm]         4
dtype: int64

In [9]:
AnamBerkane=AnamBerkane.dropna(subset=["ETP quotidien [mm]"])

**Supprimer la ligne si Humidité max < Humidité min ou viteese de vent < 0 ou Température max < Température Min**

In [10]:
for i in AnamBerkane.index:
    if(AnamBerkane["Température max"][i]<AnamBerkane["Température min"][i]):
        AnamBerkane.drop(i, axis=0, inplace=True)
        AnamBerkane.reset_index(drop=True, inplace=True)
    elif(AnamBerkane["Humidité max"][i]<AnamBerkane["Humidité min"][i]):
        AnamBerkane.drop(i, axis=0, inplace=True)
        AnamBerkane.reset_index(drop=True, inplace=True)
    elif(AnamBerkane["Vitesse du vent  moy"][i]<0):
        AnamBerkane.drop(i, axis=0, inplace=True)
        AnamBerkane.reset_index(drop=True, inplace=True)

**Convertir la date du jour au nombre de jour dans l'année**

In [11]:
for i in AnamBerkane.index:
    date_j = dt.strptime(AnamBerkane["Date/heure"][i], '%d/%m/%Y %H:%M')
    j = int(date_j.strftime("%j"))
    AnamBerkane["Date/heure"][i] = j

**Enregistrement des données nettoyées**

In [12]:
file_name = 'AnamBerkaneJr.xlsx'
AnamBerkane.to_excel(file_name)
file_name = 'EnamMksJr.xlsx'
EnamMksJr.to_excel(file_name)

# Nettoyage des données Horraires

In [13]:
z=557
l=33.80

In [14]:
def E(T):
    return 0.6108*np.exp((17.27*T)/(T+237.3))
def u2(vitesse):
    return vitesse*(4.87/(math.log(67.8*2 - 5.42)))
def ea(etmin,HR_max,etmax,HR_min):
    return (etmin*(HR_max / 100) + etmax * (HR_min / 100))/2
def ra(J):
    delta=0.409*np.sin(2*np.pi*J/365-1.39) 
    Gcs=0.0820
    phi=l*np.pi/180
    dr=1+0.033*np.cos(2*np.pi*J/365)
    ws=np.arccos(-np.tan(phi)*np.tan(delta))
    return 24*60/np.pi*Gcs*dr*(ws*np.sin(phi)*np.sin(delta)+np.cos(phi)*np.cos(delta)*np.sin(ws))
def ET_Asce(T_max,T_min,HR_max,HR_min,Rs,vitesse,J):
    Rs=Rs *86400/10**6
    U2=u2(vitesse)
    P=101.3*(((293-0.0065*z)/293)**(5.26))
    gamma=0.000665 * P
    sigma=4.903*(10**(-9))
    Tmoy=(T_max+T_min)/2
    Delta= (4098*E(Tmoy))/((Tmoy + 237.3)**2)
    
    #calcul de es
    etmin=E(T_min)
    etmax=E(T_max)
    es=(etmax+etmin)/2
    # ea
    Ea=ea(etmin,HR_max,etmax,HR_min)
    #Ra
    Ra=ra(J)
    #calcul de rso
    Rso=(0.75+2*547.6*10**(-5))*Ra
    #calcul de Rns
    Rns=0.77*Rs
    #calcul de Rnl
    Rnl=sigma*((T_max+273.16)**4+(T_min+273.16)**4)/2*(0.34-0.14*np.sqrt(Ea))*(1.35*Rs/Rso-0.35)
    #calcul de Rn
    Rn=Rns-Rnl
    #calcul de l'ET0 
    Rng=0.408*Rn
    if(Rn<0): 
        Cd=0.96
    else:
        Cd=0.24
    DT = Delta/(Delta+gamma*(1+Cd*U2))
    PT = gamma/(Delta+gamma*(1+Cd*U2))
    TT=(37/(Tmoy+273))*U2
    ETrad = DT*Rng
    ETwind = PT*TT*(es - Ea)
    et0=ETrad+ETwind
    return et0

In [15]:
EnamHorraire=pd.read_excel(r'donnee horaire ENAM LAMRIS 16 11 2021.xlsx')

**Suppression des colonnes dont on a pas besoin**

In [16]:
EnamHorraire=EnamHorraire.drop(columns=['Point De Rosée moy','Point De Rosée min','VPD moy','VPD min','Precipitation Somme','Vitesse Du Vent max Max','Direction Du Vent moy','Direction Du Vent dernier','Panneau Solaire dernier','Batterie dernier'])

In [17]:
EnamHorraire.isnull().sum()

Date/heure                   0
Température moy              4
Température max              4
Température min              4
Radiation Solaire moy        4
Humidité moy                 4
Humidité max                 4
Humidité min                 4
Vitesse du vent moy          4
Vitesse du vent max          4
ETP quotidien            33484
dtype: int64

In [18]:
EnamHorraire=EnamHorraire.dropna(subset=["Température moy","Température max","Radiation Solaire moy","Humidité moy","Humidité max","Vitesse du vent moy"])

**Supprimer la ligne si Humidité max < Humidité min ou viteese de vent < 0 ou Température max < Température Min**

In [19]:
for i in EnamHorraire.index:
    if(EnamHorraire["Humidité max"][i]<EnamHorraire["Humidité min"][i]):
        EnamHorraire.drop(i, axis=0, inplace=True)
        EnamHorraire.reset_index(drop=True, inplace=True)
    elif((EnamHorraire["Vitesse du vent moy"][i]<0) | (EnamHorraire["Vitesse du vent max"][i]<0) ):
        EnamHorraire.drop(i, axis=0, inplace=True)
        EnamHorraire.reset_index(drop=True, inplace=True)
    elif(EnamHorraire["Température max"][i]<EnamHorraire["Température min"][i]):
        EnamHorraire.drop(i, axis=0, inplace=True)
        EnamHorraire.reset_index(drop=True, inplace=True)

**Convertir la date du jour au nombre de jour dans l'année**

In [20]:
for i in EnamHorraire.index:
    date_j = dt.strptime(EnamHorraire["Date/heure"][i], '%Y-%m-%d %H:%M:%S')
    j = int(date_j.strftime("%j"))
    EnamHorraire["Date/heure"][i] = j

**Remplacer les valeurs d'ET0 null avec la valeur calculer par l'equation ASCE-PM**

In [21]:
EnamHorraire['ET Horraire']=0
for i in EnamHorraire.index:
    EnamHorraire['ET Horraire'][i]=ET_Asce(EnamHorraire["Température max"][i],EnamHorraire["Température min"][i],EnamHorraire["Humidité max"][i],EnamHorraire["Humidité min"][i],EnamHorraire["Radiation Solaire moy"][i],EnamHorraire["Vitesse du vent moy"][i],EnamHorraire["Date/heure"][i])

In [22]:
Jour=EnamHorraire["Date/heure"][0]
EnamHorraire['ET Daily calculé']=np.nan
temp=[]
case=0
for i in EnamHorraire.index:
    if (EnamHorraire["Date/heure"][i]==Jour):
        temp.append(EnamHorraire["ET Horraire"][i])
    elif(EnamHorraire["Date/heure"][i]!=Jour):
        Etdaily=sum(temp)/len(temp)
        EnamHorraire['ET Daily calculé'][case]=Etdaily
        Jour=EnamHorraire["Date/heure"][i]
        temp=[]
        temp.append(EnamHorraire["ET Horraire"][i])
        case=i-1
    if i == EnamHorraire.index[-1] :
        Etdaily=sum(temp)/len(temp)
        EnamHorraire['ET Daily calculé'][case]=Etdaily

In [23]:
EnamHorraire.isnull().sum()

Date/heure                   0
Température moy              0
Température max              0
Température min              0
Radiation Solaire moy        0
Humidité moy                 0
Humidité max                 0
Humidité min                 0
Vitesse du vent moy          0
Vitesse du vent max          0
ETP quotidien            33484
ET Horraire                  0
ET Daily calculé         33481
dtype: int64

**Nettoyage des données Ensam Meknès horraire**

In [24]:
EnsamHorraire=pd.read_excel(r'donnee horaire ENSAM 16 11 2021.xlsx')

**Suppression des colonne dont on a pas besoin**

In [25]:
EnsamHorraire=EnsamHorraire.drop(columns=['Point De Rosée moy','Point De Rosée min','VPD moy','VPD min','Precipitation Somme','Vitesse Du Vent max Max','Direction Du Vent moy','Direction Du Vent dernier','Panneau Solaire dernier','Batterie dernier','DeltaT moy','DeltaT max','DeltaT min','Humectation foliaire time'])

In [26]:
EnsamHorraire.isnull().sum()

Date/heure                   0
Température moy              1
Température max              1
Température min              1
Radiation Solaire moy        1
Humidité moy                 1
Humidité max                 1
Humidité min                 1
Vitesse du vent moy          1
Vitesse du vent max          1
ETP quotidien            20285
dtype: int64

**suppression des lignes dans les colonnes contenant des vaeurs null**

In [27]:
EnsamHorraire=EnsamHorraire.dropna(subset=["Température moy","Température max","Température min","Radiation Solaire moy","Humidité moy","Humidité max","Humidité min","Vitesse du vent moy","Vitesse du vent max"])

**Supprimer la ligne si Humidité max < Humidité min ou viteese de vent < 0 ou Température max < Température Min**

In [28]:
for i in EnsamHorraire.index:
    if(EnsamHorraire["Humidité max"][i]<EnsamHorraire["Humidité min"][i]):
        EnsamHorraire.drop(i, axis=0, inplace=True)
        EnsamHorraire.reset_index(drop=True, inplace=True)
    elif((EnsamHorraire["Vitesse du vent moy"][i]<0) | (EnsamHorraire["Vitesse du vent max"][i]<0) ):
        EnsamHorraire.drop(i, axis=0, inplace=True)
        EnsamHorraire.reset_index(drop=True, inplace=True)
    elif(EnsamHorraire["Température max"][i]<EnsamHorraire["Température min"][i]):
        EnsamHorraire.drop(i, axis=0, inplace=True)
        EnsamHorraire.reset_index(drop=True, inplace=True)

**Convertir la date du jour au nombre de jour dans l'année**

In [29]:
for i in EnsamHorraire.index:
    date_j = dt.strptime(EnsamHorraire["Date/heure"][i], '%Y-%m-%d %H:%M:%S')
    j = int(date_j.strftime("%j"))
    EnsamHorraire["Date/heure"][i] = j

**Remplacer les valeurs d'ET0 null avec la valeur calculer par l'equation ASCE-PM**

In [30]:
EnsamHorraire['ET Horraire']=0
for i in EnsamHorraire.index:
    EnsamHorraire['ET Horraire'][i]=ET_Asce(EnsamHorraire["Température max"][i],EnsamHorraire["Température min"][i],EnsamHorraire["Humidité max"][i],EnsamHorraire["Humidité min"][i],EnsamHorraire["Radiation Solaire moy"][i],EnsamHorraire["Vitesse du vent moy"][i],EnsamHorraire["Date/heure"][i])

In [31]:
Jour=EnsamHorraire["Date/heure"][0]
EnsamHorraire['ET Daily calculé']=np.nan
temp=[]
case=0
for i in EnsamHorraire.index:
    if (EnsamHorraire["Date/heure"][i]==Jour):
        temp.append(EnsamHorraire["ET Horraire"][i])
    elif(EnsamHorraire["Date/heure"][i]!=Jour):
        Etdaily=sum(temp)/len(temp)
        EnsamHorraire['ET Daily calculé'][case]=Etdaily
        Jour=EnsamHorraire["Date/heure"][i]
        temp=[]
        temp.append(EnsamHorraire["ET Horraire"][i])
        case=i-1
    if i == EnsamHorraire.index[-1] :
        Etdaily=sum(temp)/len(temp)
        EnsamHorraire['ET Daily calculé'][case]=Etdaily

In [32]:
EnsamHorraire.isnull().sum()

Date/heure                   0
Température moy              0
Température max              0
Température min              0
Radiation Solaire moy        0
Humidité moy                 0
Humidité max                 0
Humidité min                 0
Vitesse du vent moy          0
Vitesse du vent max          0
ETP quotidien            20285
ET Horraire                  0
ET Daily calculé         20284
dtype: int64

**Enregistrement des nouveaux fichier bien nettoyer**

In [33]:
file_name = 'EnamHorraire.xlsx'
EnamHorraire.to_excel(file_name)
file_name = 'EnsamHorraire.xlsx'
EnsamHorraire.to_excel(file_name)